# Exercise 2.2 - Computing Stress Fields with Tensors

### Task
Estimate the mass density given a displacement field assuming a gravity load

### Learning goals: 
To familiarize yourself with the PyTorch framework with an example from mechanics


In [ ]:
import torch
import matplotlib.pyplot as plt
from torch.autograd import grad

displacements

In [ ]:
# your code goes here: modify the displacement function from exercise 2.1 according to exercise 2.2

definition of a spatial grid

In [ ]:
nx = 5
ny = 5

x = torch.linspace(-1, 1, nx, requires_grad=True)
y = torch.linspace(-1, 1, ny, requires_grad=True)
x, y = torch.meshgrid(x, y, indexing="ij")

sampled displacement field

In [ ]:
d = torch.cat((u(x, y).unsqueeze(0), v(x, y).unsqueeze(0)), 0)
# the following is achieved by the concatenation
# d[0, :, :] == u(x,y)
# d[1, :, :] == v(x,y)

gradient computation

In [ ]:
dd_dx = torch.zeros((2, 2, nx, ny))
dd_dx[0, 0] = grad(d[0], x, torch.ones_like(x), create_graph=True, retain_graph=True)[0]
dd_dx[0, 1] = grad(d[0], y, torch.ones_like(y), create_graph=True, retain_graph=True)[0]
dd_dx[1, 0] = grad(d[1], x, torch.ones_like(x), create_graph=True, retain_graph=True)[0]
dd_dx[1, 1] = grad(d[1], y, torch.ones_like(y), create_graph=True, retain_graph=True)[0]

strain computation
$$\mathbf{\epsilon} = \frac{1}{2}(\nabla \mathbf{u} + \nabla \mathbf{u}^\intercal)$$

In [ ]:
eps = 0.5 * (dd_dx + dd_dx.permute((1, 0, 2, 3)))

material properties and 4th order material tensor


In [ ]:
E = 210000.0
nu = 0.3

C = torch.zeros((2, 2, 2, 2))  # 4th order material tensor
C[0, 0, 0, 0] = 1.0
C[0, 0, 1, 1] = nu
C[1, 1, 0, 0] = nu
C[1, 1, 1, 1] = 1.0
C[0, 1, 0, 1] = (1.0 - nu) / 2.0
C = E / (1.0 - nu ** 2) * C

stress computation
$$\mathbf{\sigma} = \mathbf{C} : \mathbf{\epsilon} $$

In [ ]:
sig = torch.tensordot(C, eps)

equilibrium computation

In [ ]:
dsig11_dx = grad(
    sig[0, 0], x, torch.ones_like(x), create_graph=True, retain_graph=True
)[0]
dsig12_dy = grad(
    sig[0, 1], y, torch.ones_like(y), create_graph=True, retain_graph=True
)[0]
dsig21_dx = grad(
    sig[1, 0], x, torch.ones_like(x), create_graph=True, retain_graph=True
)[0]
dsig22_dy = grad(
    sig[1, 1], y, torch.ones_like(y), create_graph=True, retain_graph=True
)[0]

f = torch.zeros((2, nx, ny))
f[0] = -dsig11_dx - dsig12_dy  # out of balance force in x1
f[1] = -dsig21_dx - dsig22_dy  # out of balance force in x2

density computation

In [ ]:
# your code goes here: compute the density